In [118]:
from langchain_core.prompts import PipelinePromptTemplate, PromptTemplate
from langchain_openai import ChatOpenAI
import os
import json
import random

In [119]:
file_path = 'demography.json'
demography = None
with open(file_path, 'r') as file:
    demography = json.load(file)
demography

{'age': [{'category': 'Teens', 'start': 13, 'end': 19},
  {'category': 'Young Adults', 'start': 20, 'end': 35},
  {'category': 'Adults', 'start': 35, 'end': 50},
  {'category': 'Seniors', 'start': 51, 'end': 70}],
 'genders': ['Female', 'Male', 'Non-binary'],
 'ethnicities': ['Hispanic/Latino',
  'Caucasian',
  'Asian',
  'African American',
  'Mixed race',
  'Native American'],
 'locations': ['North America',
  'Europe',
  'Asia',
  'South America',
  'Africa',
  'Oceania'],
 'professionalBackgrounds': ['Blogger',
  'Professional Critic',
  'Film Student',
  'Casual Moviegoer',
  'YouTuber',
  'Industry Professional',
  'IT Professional',
  'Marketing Professional',
  'Teacher',
  'Medical Professional',
  'Literature Teacher',
  'History Teacher'],
 'educations': ['High School',
  "Bachelor's Degree",
  "Master's Degree",
  'Doctorate',
  'Self-taught'],
 'incomeLevels': ['Low Income', 'Middle Income', 'High Income']}

In [120]:
def get_random_persona(demography, seed=None):
    if seed:
        random.seed(seed)
    while True:
        permutation = {}
        permutation['age'] = random.choice(demography['age'])
        permutation['gender'] = random.choice(demography['genders'])
        permutation['ethnicity'] = random.choice(demography['ethnicities'])
        permutation['location'] = random.choice(demography['locations'])
        permutation['professionalBackground'] = random.choice(demography['professionalBackgrounds'])
        permutation['education'] = random.choice(demography['educations'])
        permutation['incomeLevel'] = random.choice(demography['incomeLevels'])
        yield permutation

In [121]:
random_persona_generator = get_random_persona(demography, 42)
next(random_persona_generator)

{'age': {'category': 'Teens', 'start': 13, 'end': 19},
 'gender': 'Female',
 'ethnicity': 'Native American',
 'location': 'Asia',
 'professionalBackground': 'Casual Moviegoer',
 'education': "Bachelor's Degree",
 'incomeLevel': 'Low Income'}

In [122]:
individual_prompt = PromptTemplate.from_template("""
You're an individual with a specific age, gender, or ethnic background from a defined geographic location.

Those parameters are specified within the following block.
It starts with "BEGIN demography" and ends with "END demography".

BEGIN demography
age range start: {ageStart}
age range end: {ageEnd}
gender: {gender}
ethnicity: {ethnicity}
location: {location}
professional background: {profession}
education: {education}
income: {income}
END demography
""")

In [123]:
simple_review_prompt = PromptTemplate.from_template("""
Write a review about Witcher season 2? 
""")

In [138]:
project_prompt = PromptTemplate.from_template("""
Below are high-level movie or TV show ideas and some essential parameters.
It starts with "BEGIN content" and ends with "END content".

BEGIN content
name: {name}
type: {type}
cast: {cast}
budget: {budget}
END content
""")

In [154]:
cta_prompt = PromptTemplate.from_template("""
Write a simple review as someone who just watched the {name}.
Highlight points that you liked and didn't like.
Be honest, your feedback is valuable and will be applied by a studio to improve the content.
""")

In [155]:
full_prompt = individual_prompt + project_prompt + cta_prompt

In [156]:
model=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-1106", openai_api_key=os.getenv("OPENAI_API_KEY"))

In [157]:
def generate_demography_prompt_input(persona):
    return {
      "ageStart": persona["age"]["start"],
      "ageEnd": persona["age"]["end"],
      "gender": persona["gender"],
      "ethnicity": persona["ethnicity"],
      "location": persona["location"],
      "profession": persona["professionalBackground"],
      "education": persona["education"],
      "income": persona["incomeLevel"]
    }

In [158]:
def generate_content_prompt_input(content):
    return {
        "name": content["name"],
        "type": content["type"],
        "cast": ",".join(content["cast"]),
        "budget": content["budget"]
    }

In [159]:
# prompt generator
def generate_prompt_input(persona, content):
    return {
        **generate_demography_prompt_input(persona),
        **generate_content_prompt_input(content)
    }

In [160]:
# prompt testing
persona = next(random_persona_generator)
content = {
    "name": "The witcher movie",
    "type": "Movie",
    "cast": ["Chris Hemsworth", "Natalia Oreiro"],
    "budget": 4_000_000
}
prompt_input = generate_prompt_input(persona, content)

In [161]:
full_prompt.invoke(prompt_input)

StringPromptValue(text='\nYou\'re an individual with a specific age, gender, or ethnic background from a defined geographic location.\n\nThose parameters are specified within the following block.\nIt starts with "BEGIN demography" and ends with "END demography".\n\nBEGIN demography\nage range start: 13\nage range end: 19\ngender: Male\nethnicity: Hispanic/Latino\nlocation: Asia\nprofessional background: Industry Professional\neducation: Self-taught\nincome: Middle Income\nEND demography\n\nBelow are high-level movie or TV show ideas and some essential parameters.\nIt starts with "BEGIN content" and ends with "END content".\n\nBEGIN content\nname: The witcher movie\ntype: Movie\ncast: Chris Hemsworth,Natalia Oreiro\nbudget: 4000000\nEND content\n\nWrite a simple review as someone who just watched the The witcher movie.\nHighlight points that you liked and didn\'t like.\nBe honest, your feedback is valuable and will be applied by a studio to improve the content.\n')

In [163]:
chain = full_prompt | model 

In [164]:
chain.invoke(prompt_input)

AIMessage(content='As a Hispanic/Latino male in the 13-19 age range from Asia, I just watched The Witcher movie and I have to say, I was thoroughly impressed. The action sequences were intense and well choreographed, and Chris Hemsworth and Natalia Oreiro delivered fantastic performances. The special effects were top-notch, and the overall production quality was excellent. The storyline kept me engaged throughout the movie, and I appreciated the attention to detail in bringing the world of The Witcher to life.\n\nHowever, I did feel that the pacing of the movie was a bit rushed at times, and some character development felt lacking. As a fan of the original source material, I would have liked to see a deeper exploration of the lore and mythology of The Witcher universe. Additionally, I felt that the budget constraints may have limited the scope of the movie, and certain scenes could have benefited from a larger investment.\n\nOverall, I thoroughly enjoyed The Witcher movie and would lov